# Trying LR to see Correlations, T-Values, etc.

Multi LR
    - how do the t-values look?

Forward/Backwards Selection (how does adding an additional var affect things)

VIF (basically, is this variable explained by LR of the others --> probably low though)

Spam FE

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [6]:
DIRECTORY = './data/'
train = pd.read_csv(DIRECTORY + 'train.csv').drop(columns='id')
# train['FloodProbability'] = np.round(train['FloodProbability'] * 200 - 56, decimals=0).astype(int)

def downcast_df(df):
    for col in df.select_dtypes(include=["int", "float"]).columns:
        if df[col].dtype == "int64" or df[col].dtype == "int32":
            df[col] = pd.to_numeric(df[col], downcast="integer")
        elif df[col].dtype == "float64" or df[col].dtype == "float32":
            df[col] = pd.to_numeric(df[col], downcast="float")
    return df

# Example usage
train = downcast_df(train)

In [7]:
model = LinearRegression()
train = train.sample(frac=1, random_state=40).reset_index(drop=True)
non_extreme = train[(train.drop(columns='FloodProbability') <= 12).sum(axis=1).astype(bool)]
score = cross_val_score(model, X=non_extreme.drop(columns='FloodProbability'), y=non_extreme['FloodProbability'], cv=5, scoring=make_scorer(r2_score))
score

array([0.84590348, 0.84558981, 0.84496012, 0.84332034, 0.84493074])

In [9]:
# Manual CV w/ statsmodels for better viz

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(train)):
    train_fold = train.iloc[train_idx]
    test_fold = train.iloc[test_idx]
    X_train, y_train = train_fold.drop(columns='FloodProbability').mean(axis=1), train_fold['FloodProbability']
    X_test, y_test = test_fold.drop(columns='FloodProbability').mean(axis=1), test_fold['FloodProbability']

    model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
    predictions = model.predict(sm.add_constant(X_test))

    # TSS = np.var(y_test)
    # RSS = np.var(y_test - predictions)
    # score = (TSS - RSS)/TSS
    score = r2_score(y_test, predictions)
    print(fold_idx, score)
    # print(model.summary())

    train_predictions = model.predict(sm.add_constant(X_train))
    train_score = r2_score(y_train, train_predictions)
    print(train_score)

    # sns.scatterplot(x=y_test, y=(predictions - y_test)**2, alpha=0.1)
    # plt.show()


0 0.8449467155390502
0.8449349665035999
1 0.8464680623071348
0.8445527331060134
2 0.8442302043947386
0.845113104992887
3 0.8448549423711544
0.844958049403467
4 0.8441718570497602
0.8451283634712662


In [4]:
# Mean prediction

model = sm.OLS(train['FloodProbability'], train.drop(columns='FloodProbability').mean(axis=1)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       FloodProbability   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          6.773e+08
Date:                Wed, 05 Feb 2025   Prob (F-statistic):                        0.00
Time:                        13:45:40   Log-Likelihood:                      2.7550e+06
No. Observations:             1117957   AIC:                                 -5.510e+06
Df Residuals:                 1117956   BIC:                                 -5.510e+06
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1022   3.93e-06    2.6e+04      0.000       0.102       0.102
==============================================================================
Omnibus:                    42062.631   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            50477.179
Skew:                           0.441   Prob(JB):                         0.00
Kurtosis:                       3.553   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
r2_score(train['FloodProbability'], train.drop(columns='FloodProbability').mean(axis=1) * 0.1022)

0.8372797463161574